<br>

# 6) Instruction finetuning (part 2; finetuning)

- In this notebook, we get to the actual finetuning part
- But first, let's briefly introduce a technique, called LoRA, that makes the finetuning more efficient
- It's not required to use LoRA, but it can result in noticeable memory savings while still resulting in good modeling performance

<br>

# 6.1 Introduction to LoRA

- Low-rank adaptation (LoRA) is a machine learning technique that modifies a pretrained model to better suit a specific, often smaller, dataset by adjusting only a small, low-rank subset of the model's parameters
- This approach is important because it allows for efficient finetuning of large models on task-specific data, significantly reducing the computational cost and time required for finetuning

- Suppose we have a large weight matrix $W$ for a given layer
- During backpropagation, we learn a $\Delta W$ matrix, which contains information on how much we want to update the original weights to minimize the loss function during training
- In regular training and finetuning, the weight update is defined as follows:

$$W_{\text{updated}} = W + \Delta W$$

- The LoRA method proposed by [Hu et al.](https://arxiv.org/abs/2106.09685) offers a more efficient alternative to computing the weight updates $\Delta W$ by learning an approximation of it, $\Delta W \approx AB$.
- In other words, in LoRA, we have the following, where $A$ and $B$ are two small weight matrices:

$$W_{\text{updated}} = W + AB$$

- The figure below illustrates these formulas for full finetuning and LoRA side by side

<img src="figures/08.png" width="1100px">

- If you paid close attention, the full finetuning and LoRA depictions in the figure above look slightly different from the formulas I have shown earlier
- That's due to the distributive law of matrix multiplication: we don't have to add the weights with the updated weights but can keep them separate
- For instance, if $x$ is the input data, then we can write the following for regular finetuning:

$$x (W+\Delta W) = x W + x \Delta W$$

- Similarly, we can write the following for LoRA:

$$x (W+A B) = x W + x A B$$

- The fact that we can keep the LoRA weight matrices separate makes LoRA especially attractive
- In practice, this means that we don't have to modify the weights of the pretrained model at all, as we can apply the LoRA matrices on the fly
- After setting up the dataset and loading the model, we will implement LoRA in the code to make these concepts less abstract

<img src="figures/09.png" width="800px">

<br>
<br>
<br>
<br>

# 6.2 Creating training and test sets

- There's one more thing before we can start finetuning: creating the training and test subsets
- We will use 85% of the data for training and the remaining 15% for testing

In [3]:
import json


file_path = "../06_finetuning/instruction-data.json"

with open(file_path, "r") as file:
    data = json.load(file)
print("Number of entries:", len(data))

Number of entries: 1100


In [4]:
train_portion = int(len(data) * 0.85)  # 85% for training
test_portion = int(len(data) * 0.15)    # 15% for testing

train_data = data[:train_portion]
test_data = data[-test_portion:] # or another way: test_data = data[train_portion:]

In [5]:
print("Training set length:", len(train_data))
print("Test set length:", len(test_data))

Training set length: 935
Test set length: 165


In [10]:
with open("train.json", "w") as json_file:
    json.dump(train_data, json_file, indent=4)
    
with open("test.json", "w") as json_file:
    json.dump(test_data, json_file, indent=4)

<br>
<br>

# 6.3 Instruction finetuning

- Using LitGPT, we can finetune the model via `litgpt finetune model_dir`
- However, here, we will use LoRA finetuning `litgpt finetune_lora model_dir` since it will be quicker and less resource intensive

In [42]:
# !litgpt finetune_lora microsoft/phi-1_5 --data JSON --data.val_split_fraction 0.1 --data.json_path train.json --train.epochs 1 --train.log_interval 10
!litgpt finetune_lora microsoft/phi-1_5 \
--devices 1 \
--data JSON \
--data.val_split_fraction 0.1 \
--data.json_path train.json \
--train.epochs 1 \
--train.max_steps 10 \
--train.log_interval 1

^C


C:\python397\lib\site-packages\litgpt\args.py:49: UserWarning: `--train.lr_warmup_steps` should be less than `--train.max_steps`. Got 100 lr_warmup_steps and 10 max_steps.
  warnings.warn(
Using bfloat16 Automatic Mixed Precision (AMP)
Seed set to 1337


uvloop is not installed. Falling back to the default asyncio event loop. Please install uvloop for better performance using `pip install uvloop`.
uvloop is not installed. Falling back to the default asyncio event loop.
{'access_token': None,
 'checkpoint_dir': WindowsPath('checkpoints/microsoft/phi-1_5'),
 'data': JSON(json_path=WindowsPath('train.json'), mask_prompt=False, val_split_fraction=0.1, prompt_style=<litgpt.prompts.Alpaca object at 0x000002499FC064F0>, ignore_index=-100, seed=42, num_workers=4),
 'devices': 1,
 'eval': EvalArgs(interval=100, max_new_tokens=100, max_iters=100, initial_validation=False, final_validation=True, evaluate_example='first'),
 'logger_name': 'csv',
 'lora_alpha': 16,
 'lora_dropout': 0.05,
 'lora_head': False,
 'lora_key': False,
 'lora_mlp': False,
 'lora_projection': False,
 'lora_query': True,
 'lora_r': 8,
 'lora_value': True,
 'num_nodes': 1,
 'optimizer': 'AdamW',
 'out_dir': WindowsPath('out/finetune/lora'),
 'precision': None,
 'quantize': No

<br>
<br>
<br>
<br>

# Exercise 1: Generate and save the test set model responses of the base model

- In this excercise, we are collecting the model responses on the test dataset so that we can evaluate them later


- Starting with the original model before finetuning, load the model using the LitGPT Python API (`LLM.load` ...)
- Then use the `LLM.generate` function to generate the responses for the test data
- The following utility function will help you to format the test set entries as input text for the LLM

In [6]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

print(format_input(test_data[0]))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the sentence using a simile.

### Input:
The car is very fast.


In [ ]:
from litgpt import LLM
# if you download a new model then first do: del llm
llm = LLM.load("microsoft/phi-1_5")

In [7]:
# from tqdm import tqdm

# for i in tqdm(range(len(test_data))):
#     response = llm.generate(test_data[i])
#     test_data[i]["base_model"] = response

for i in range(len(test_data)):
    response = llm.generate(format_input(test_data[i]))
    test_data[i]["base_model"] = response

KeyboardInterrupt: 

- Using this utility function, generate and save all the test set responses generated by the model and add them to the `test_set`
- For example, if `test_data[0]` entry is as follows before:
    
```
{'instruction': 'Rewrite the sentence using a simile.',
 'input': 'The car is very fast.',
 'output': 'The car is as fast as lightning.'}
```

- Modify the `test_data` entry so that it contains the model response:
    
```
{'instruction': 'Rewrite the sentence using a simile.',
 'input': 'The car is very fast.',
 'output': 'The car is as fast as lightning.',
 'base_model': 'The car is as fast as a cheetah sprinting across the savannah.'
}
```

- Do this for all test set entries, and then save the modified `test_data` dictionary as `test_base_model.json`


In [19]:
test_data[6]

{'instruction': 'Generate a humorous anecdote.',
 'input': '',
 'output': 'Why was the math book sad? Because it had too many problems!',
 'base_model': "\n\n```python\nimport requests\n\nresponse = requests.get('https://api.geographer.com/humor')\n\nif response.status_code == 200:\n  print(response.text)\nelse:"}

<br>
<br>
<br>
<br>

# Exercise 2: Generate and save the test set model responses of the finetuned model

- Repeat the steps from the previous exercise but this time collect the responses of the finetuned model
- Save the resulting `test_data` dictionary as `test_base_and_finetuned_model.json`

<br>
<br>
<br>
<br>

# Solution

In [20]:
from litgpt import LLM

del llm
llm2 = LLM.load("out/finetune/lora/final/")

c:\python397\lib\site-packages\litgpt\utils.py:613: UserWarning: The file size of out\finetune\lora\final\lit_model.pth is over 4.2 GB. Using a model with more than 1B parameters on a CPU can be slow, it is recommended to switch to a GPU.
  warnings.warn(


In [25]:
from tqdm import tqdm

for i in tqdm(range(6)):
    response = llm2.generate(format_input(test_data[i]))
    test_data[i]["finetuned_model"] = response

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [20:22<00:00, 203.73s/it]


In [26]:
for i in range(6):
    print(test_data[i])

{'instruction': 'Rewrite the sentence using a simile.', 'input': 'The car is very fast.', 'output': 'The car is as fast as lightning.', 'base_model': "\nThe car is as fast as a cheetah.\n```python\n# The first step is to find appropriate similes for each word.\n# For the sentence 'The car is very fast.'\n# The similes will compare", 'finetuned_model': 'The car is as fast as a cheetah.'}
{'instruction': 'What type of cloud is typically associated with thunderstorms?', 'input': '', 'output': 'The type of cloud typically associated with thunderstorms is cumulonimbus.', 'base_model': '\n```python\nimport flask\n\napp = flask.Flask(__name__)\n\n@app.route(\'/rainstorm\')\ndef rainstorm():\n    return "A thunderstorm is typically associated with rain."\n', 'finetuned_model': 'Tornado clouds are typically associated with thunderstorms.'}
{'instruction': "Name the author of 'Pride and Prejudice'.", 'input': '', 'output': 'Jane Austen.', 'base_model': "\n## Solution\n'Jane Austen'\n\n## Overall

In [27]:
with open("test_base_and_finetuned_model.json", "w") as json_file:
    json.dump(test_data[:6], json_file, indent=4)